In [2]:
%load_ext lab_black

import pandas as pd

In [ ]:
traffic = (
    pd.read_parquet("/data/data/open_dataset/oag/2018.parquet")
    .groupby(["GeneralAcft", "DepAirport", "ArrAirport"])[["Km"]]
    .agg({"Km": ["first", "count"]})
    .reset_index()
)
traffic.columns = ["".join(a) for a in traffic.columns.to_flat_index()]
traffic = traffic.rename(
    columns={
        "GeneralAcft": "typecode",
        "DepAirport": "origin",
        "ArrAirport": "destination",
        "Kmfirst": "km",
        "Kmcount": "flights",
    }
).sort_values("flights", ascending=False)
traffic.to_parquet("traffic_raw.parquet")
print(sum(traffic.flights), "flights")
traffic

In [23]:
traffic = pd.read_parquet("traffic_raw.parquet")
print(sum(traffic.flights), "flights")
traffic

47282699 flights


,typecode,origin,destination,km,flights
130554,TRN,LHR,QQP,19,68544
130641,TRN,QQP,LHR,19,66921
130957,TRN,XRE,QQP,55,43062
130642,TRN,QQP,XRE,55,42714
131171,TRN,ZSB,LZS,106,28478
...,...,...,...,...,...
36167,340,CDG,FIH,6062,1
36173,340,CDG,JIB,5589,1
55282,737,LPA,POR,4624,1
55259,737,LPA,LPP,4884,1


In [24]:
# Airports codes: iata -> icao
traffic = (
    traffic.merge(
        pd.read_csv("routes.csv"),
        left_on=["origin", "destination"],
        right_on=["origin_iata", "destination_iata"],
        how="left",
    )[
        [
            "typecode",
            "origin_icao",
            "destination_icao",
            "km_x",
            "flights",
        ]
    ]
    .rename(
        columns={
            "origin_icao": "origin",
            "destination_icao": "destination",
            "km_x": "km",
        }
    )
    .dropna()
).sort_values("flights", ascending=False)
print(sum(traffic.flights), "flights")
traffic

44403574 flights


,typecode,origin,destination,km,flights
6,737,RKSS,RKPC,451,23983
7,737,RKPC,RKSS,451,23625
14,737,YMML,YSSY,705,19385
15,737,YSSY,YMML,705,19361
16,EC3,LNMC,LFMN,18,19019
...,...,...,...,...,...
126687,310,LPLA,CYHM,4387,1
126686,310,LPLA,LPPT,1553,1
126685,ERJ,KCMH,KBMG,332,1
126684,737,GMMX,GMFO,669,1


In [25]:
# Aircraft typecodes: remove "TBR" codes (bus, train, helicopters...)
traffic = (
    traffic.set_index("typecode")
    .drop(
        index=pd.read_csv("typecodes.csv").query("icao=='TBR'").typecode.to_list(),
        errors="ignore",
    )
    .reset_index()
)
print(sum(traffic.flights), "flights")

# Aircraft typecodes: iata -> icao
traffic = (
    traffic.merge(pd.read_csv("typecodes.csv"), how="left", on="typecode")
    .rename(columns={"typecode": "typecode_oag", "icao": "typecode_icao"})[
        ["typecode_oag", "typecode_icao", "origin", "destination", "km", "flights"]
    ]
    .sort_values("flights", ascending=False)
)
traffic.to_parquet("traffic.parquet")
traffic

38427575 flights


,typecode_oag,typecode_icao,origin,destination,km,flights
0,737,B38M,RKSS,RKPC,451,23983
6,737,B736,RKSS,RKPC,451,23983
9,737,B739,RKSS,RKPC,451,23983
8,737,B738,RKSS,RKPC,451,23983
7,737,B737,RKSS,RKPC,451,23983
...,...,...,...,...,...,...
543066,737,B737,HAAB,HHSB,620,1
543065,737,B736,HAAB,HHSB,620,1
543064,737,B735,HAAB,HHSB,620,1
543063,737,B734,HAAB,HHSB,620,1


In [26]:
# Filter out flights with invalid GC distances (execeding max flight distance of a specific aircraft type)
import numpy as np
from scipy import stats

traffic = pd.read_parquet("traffic.parquet")
traffic["z_score"] = np.NAN
typecode = "typecode_icao"

for ac in traffic[typecode].unique():
    traffic.loc[traffic[typecode] == ac, "z_score"] = abs(
        stats.zscore(traffic.loc[traffic[typecode] == ac, "km"])
    )
filtered = traffic.loc[(traffic.z_score < 5) | (traffic.z_score.isna())]
removed = traffic.loc[traffic.z_score > 5]
print(
    "{:,.0f} ({:.2f}% of total) routes removed".format(
        len(removed), len(removed) / len(traffic) * 100
    )
)

# for ac in filtered[typecode].unique():
#     filtered.loc[filtered[typecode] == ac, "z_score"] = abs(
#         stats.zscore(filtered.loc[filtered[typecode] == ac, "km"])
#     )
# filtered = filtered.loc[(filtered.z_score < 6) | (filtered.z_score.isna())]
# removed = filtered.loc[traffic.z_score > 6]
# print(
#     "{:,.0f} ({:.2f}% of total) routes removed".format(
#         len(removed), len(removed) / len(traffic) * 100
#     )
# )
traffic = filtered.reset_index(drop=True).drop("z_score", axis=1)
# # traffic.to_parquet("traffic.parquet")
traffic

500 (0.09% of total) routes removed


,typecode_oag,typecode_icao,origin,destination,km,flights
0,737,B38M,RKSS,RKPC,451,23983
1,737,B736,RKSS,RKPC,451,23983
2,737,B739,RKSS,RKPC,451,23983
3,737,B738,RKSS,RKPC,451,23983
4,737,B737,RKSS,RKPC,451,23983
...,...,...,...,...,...,...
564475,737,B737,HAAB,HHSB,620,1
564476,737,B736,HAAB,HHSB,620,1
564477,737,B735,HAAB,HHSB,620,1
564478,737,B734,HAAB,HHSB,620,1


### Fuel computation method 1 : typecode distribution per route not weighted with historical data

In [27]:
fuel = (
    traffic.merge(
        pd.read_csv("ac_model_coefficients.csv", index_col=[0]),
        how="left",
        left_on="typecode_icao",
        right_on="ac_code_icao",
    )
    .drop(
        ["ac_code_icao", "wake", "reduced_fuel_score", "reduced_sample_size"],
        axis=1,
    )
    .assign(
        fuel_kg_flight=lambda x: x.reduced_fuel_a1 * x.km**2
        + x.reduced_fuel_a2 * x.km
        + x.reduced_fuel_intercept,
        fuel_kg=lambda x: x.fuel_kg_flight * x.flights,
    )
    .sort_values("fuel_kg", ascending=False)
)
display(fuel)
fuel = (
    fuel.groupby(["typecode_oag", "origin", "destination", "e_type"])[
        ["km", "flights", "fuel_kg"]
    ]
    .agg({"fuel_kg": ["min", "max", "mean"]})
    .reset_index()
    .sort_values(("fuel_kg", "mean"), ascending=False)
)
fuel.columns = ["".join(a) for a in fuel.columns.to_flat_index()]
fuel.to_parquet("fuel1.parquet")
display(fuel)

,typecode_oag,typecode_icao,origin,destination,km,flights,e_type,reduced_fuel_a1,reduced_fuel_a2,reduced_fuel_intercept,fuel_kg_flight,fuel_kg
5314,74F,B744,VHHX,PANC,8163,3362,Jet,0.000330,8.335664,5173.387530,95196.193639,3.200496e+08
27790,380,A388,EGLL,WSSS,10876,1435,Jet,0.000298,11.402874,6408.219743,165632.342916,2.376824e+08
27851,380,A388,WSSS,EGLL,10876,1434,Jet,0.000298,11.402874,6408.219743,165632.342916,2.375168e+08
10976,74F,B744,PANC,VHHX,8163,2390,Jet,0.000330,8.335664,5173.387530,95196.193639,2.275189e+08
3589,74F,B744,PANC,KORD,4566,3939,Jet,0.000330,8.335664,5173.387530,50110.662703,1.973859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
532779,CNA,C172,BZ-SVK,BZ-TZA,89,1,Piston,0.000001,0.117295,11.483789,21.932851,2.193285e+01
537951,CNA,C172,FALD,FANG,48,1,Piston,0.000001,0.117295,11.483789,17.116802,1.711680e+01
538510,CNA,C172,PAFM,PAOB,47,1,Piston,0.000001,0.117295,11.483789,16.999389,1.699939e+01
533616,CNA,C172,ZA-0069,FANG,42,1,Piston,0.000001,0.117295,11.483789,16.412363,1.641236e+01


,typecode_oag,origin,destination,e_type,fuel_kgmin,fuel_kgmax,fuel_kgmean
71389,74F,VHHX,PANC,Jet,3.200496e+08,3.200496e+08,3.200496e+08
37289,380,EGLL,WSSS,Jet,2.376824e+08,2.376824e+08,2.376824e+08
37498,380,WSSS,EGLL,Jet,2.375168e+08,2.375168e+08,2.375168e+08
71097,74F,PANC,VHHX,Jet,2.275189e+08,2.275189e+08,2.275189e+08
71089,74F,PANC,KORD,Jet,1.973859e+08,1.973859e+08,1.973859e+08
...,...,...,...,...,...,...,...
90077,CNA,BZ-SVK,BZ-TZA,Piston,2.193285e+01,2.193285e+01,2.193285e+01
90149,CNA,FALD,FANG,Piston,1.711680e+01,1.711680e+01,1.711680e+01
90958,CNA,PAFM,PAOB,Piston,1.699939e+01,1.699939e+01,1.699939e+01
91450,CNA,ZA-0069,FANG,Piston,1.641236e+01,1.641236e+01,1.641236e+01


In [28]:
fuel_e_type = (fuel.groupby("e_type").sum() / 1e9).round(2).reset_index()
fuel_e_type

,e_type,fuel_kgmin,fuel_kgmax,fuel_kgmean
0,Jet,233.98,292.88,259.02
1,Piston,0.03,0.03,0.03
2,Turboprop,2.07,2.91,2.36


In [29]:
# FEAT paper 257 Mt (see page 10 FEAT paper)
fuel_e_type.sum()

e_type         JetPistonTurboprop
fuel_kgmin                 236.08
fuel_kgmax                 295.82
fuel_kgmean                261.41
dtype: object

In [30]:
# Jets burn jet A-1 fuel (3.16 Kg CO2/kg fuel, Piston 3.10 Kg CO2/kg fuel)
jet, piston = 3.16, 3.10
cumul = []
for e_type in ["Jet", "Piston"]:
    cumul.append(
        fuel.query(f"e_type=='{e_type}'").assign(
            co2=lambda x: x.fuel_kgmean * 3.16 if e_type == "Jet" else 3.10
        )
    )
co2 = pd.concat(cumul)
co2

,typecode_oag,origin,destination,e_type,fuel_kgmin,fuel_kgmax,fuel_kgmean,co2
71389,74F,VHHX,PANC,Jet,3.200496e+08,3.200496e+08,3.200496e+08,1.011357e+09
37289,380,EGLL,WSSS,Jet,2.376824e+08,2.376824e+08,2.376824e+08,7.510764e+08
37498,380,WSSS,EGLL,Jet,2.375168e+08,2.375168e+08,2.375168e+08,7.505530e+08
71097,74F,PANC,VHHX,Jet,2.275189e+08,2.275189e+08,2.275189e+08,7.189597e+08
71089,74F,PANC,KORD,Jet,1.973859e+08,1.973859e+08,1.973859e+08,6.237394e+08
...,...,...,...,...,...,...,...,...
90077,CNA,BZ-SVK,BZ-TZA,Piston,2.193285e+01,2.193285e+01,2.193285e+01,3.100000e+00
90149,CNA,FALD,FANG,Piston,1.711680e+01,1.711680e+01,1.711680e+01,3.100000e+00
90958,CNA,PAFM,PAOB,Piston,1.699939e+01,1.699939e+01,1.699939e+01,3.100000e+00
91450,CNA,ZA-0069,FANG,Piston,1.641236e+01,1.641236e+01,1.641236e+01,3.100000e+00


In [31]:
co2_e_type = (co2.groupby("e_type").sum() / 1e9).round(10).reset_index()
co2_e_type

,e_type,fuel_kgmin,fuel_kgmax,fuel_kgmean,co2
0,Jet,233.984932,292.877353,259.024962,818.518881
1,Piston,0.028147,0.028147,0.028147,0.000006


In [32]:
# FEAT paper 812Mt, IATA 833-905 Mt, ICCT 848 Mt (see page 10 FEAT paper)
print("Total CO2:", int(co2_e_type["co2"].sum()), "Mt")

Total CO2: 818 Mt


### Fuel computation method 2 : typecode distribution per route weighted with historical ADS-B data

In [33]:
traffic = (
    traffic.merge(
        pd.read_parquet("distribution.parquet"),
        how="left",
        left_on=["typecode_icao", "origin", "destination"],
        right_on=["typecode", "origin", "destination"],
    )
    .drop("typecode", axis=1)
    .sort_values(["origin", "destination"], ascending=True)
)
traffic

,typecode_oag,typecode_icao,origin,destination,km,flights,ratio
358908,D28,D228,03N,PKMJ,488,53,NaN
81675,CNA,C172,0AK,PABE,140,666,NaN
81683,CNA,C172,0AK,PAMO,43,666,NaN
29654,CNA,C172,16A,4A2,10,1387,NaN
125265,CNA,C172,16A,PABE,35,390,NaN
...,...,...,...,...,...,...,...
171396,737,B38M,rjns,ZSPD,1577,300,NaN
338530,32S,A321,rjns,ZSPD,1577,65,NaN
338531,32S,A320,rjns,ZSPD,1577,65,NaN
338532,32S,A319,rjns,ZSPD,1577,65,NaN


In [34]:
ratios = (
    traffic.groupby(["typecode_oag", "origin", "destination"])[["ratio"]]
    .agg({"ratio": ["sum", "count", lambda x: x.isnull().sum()]})
    .rename(columns={"<lambda_0>": "nans"})
    .reset_index()
)
ratios.columns = ["".join(a) for a in ratios.columns.to_flat_index()]
ratios

,typecode_oag,origin,destination,ratiosum,ratiocount,rationans
0,100,AGGH,AYPY,0.0,0,1
1,100,AGGH,NFFN,0.0,0,1
2,100,AGGH,NVVV,0.0,0,1
3,100,AYBK,AYIQ,0.0,0,1
4,100,AYBK,AYPY,0.0,0,1
...,...,...,...,...,...,...
118371,YN2,SKUI,SKNQ,0.0,0,1
118372,YN7,FVBU,FVHA,0.0,0,1
118373,YN7,FVFA,FVHA,0.0,0,1
118374,YN7,FVHA,FVBU,0.0,0,1


In [35]:
traffic = traffic.merge(
    ratios, how="left", on=["typecode_oag", "origin", "destination"]
)

traffic.loc[(traffic.ratiosum <= 1) & (traffic.rationans == 0), "ratio"] = (
    traffic.ratio + (1 - traffic.ratiosum) / traffic.ratiocount
)
traffic.loc[(traffic.ratio.isnull()), "ratio"] = (
    1 - traffic.ratiosum
) / traffic.rationans
traffic.loc[:, "flights"] = traffic.flights * traffic.ratio

# method1 : 38429915 flights
print(int(sum(traffic.flights)), "flights")

38425971 flights


In [36]:
fuel = (
    traffic.merge(
        pd.read_csv("ac_model_coefficients.csv", index_col=[0]),
        how="left",
        left_on="typecode_icao",
        right_on="ac_code_icao",
    )
    .drop(
        ["ac_code_icao", "wake", "reduced_fuel_score", "reduced_sample_size"],
        axis=1,
    )
    .assign(
        fuel_kg_flight=lambda x: x.reduced_fuel_a1 * x.km**2
        + x.reduced_fuel_a2 * x.km
        + x.reduced_fuel_intercept,
        fuel_kg=lambda x: x.fuel_kg_flight * x.flights,
    )
    .sort_values("fuel_kg", ascending=False)
)
fuel.to_parquet("fuel2.parquet")
fuel

,typecode_oag,typecode_icao,origin,destination,km,flights,ratio,ratiosum,ratiocount,rationans,e_type,reduced_fuel_a1,reduced_fuel_a2,reduced_fuel_intercept,fuel_kg_flight,fuel_kg
462809,74F,B744,VHHX,PANC,8163,3.362000e+03,1.000000e+00,0.000000,0,1,Jet,0.000330,8.335664,5173.387530,95196.193639,3.200496e+08
68215,380,A388,EGLL,WSSS,10876,1.435000e+03,1.000000e+00,0.252605,1,0,Jet,0.000298,11.402874,6408.219743,165632.342916,2.376824e+08
490451,380,A388,WSSS,EGLL,10876,1.434000e+03,1.000000e+00,0.258364,1,0,Jet,0.000298,11.402874,6408.219743,165632.342916,2.375168e+08
379262,74F,B744,PANC,VHHX,8163,2.390000e+03,1.000000e+00,0.000000,0,1,Jet,0.000330,8.335664,5173.387530,95196.193639,2.275189e+08
378872,74F,B744,PANC,KORD,4566,3.939000e+03,1.000000e+00,0.463215,1,0,Jet,0.000330,8.335664,5173.387530,50110.662703,1.973859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153821,737,B736,KCMH,KOAK,3386,-9.547918e-15,-3.172066e-17,1.000000,3,7,Jet,0.000060,2.390726,1141.073187,9921.574753,-9.473038e-11
153822,737,B735,KCMH,KOAK,3386,-9.547918e-15,-3.172066e-17,1.000000,3,7,Jet,0.000074,2.644616,1144.582138,10949.401982,-1.045440e-10
153824,737,B733,KCMH,KOAK,3386,-9.547918e-15,-3.172066e-17,1.000000,3,7,Jet,-0.000019,3.397847,783.830826,12075.792357,-1.152987e-10
153818,737,B739,KCMH,KOAK,3386,-9.547918e-15,-3.172066e-17,1.000000,3,7,Jet,0.000059,3.080558,1193.958284,12297.251987,-1.174132e-10


In [37]:
fuel_e_type = (
    (fuel.groupby("e_type")[["fuel_kg"]].sum() / 1e9)
    .round(2)
    .reset_index()
    .rename(columns={"fuel_kg": "fuel_Mt"})
)
fuel_e_type

,e_type,fuel_Mt
0,Jet,259.79
1,Piston,0.03
2,Turboprop,2.43


In [38]:
# FEAT paper 257 Mt (see page 10 FEAT paper), Method 1: 236 min, 295 max, 261 mean
int(sum(fuel_e_type.fuel_Mt))

262